# Discovering Sentinel-2 products

In [2]:
%load_ext autoreload
%autoreload 2

from eo_tools.util import visualize_S2_products 

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import json

import geopandas as gpd
import pandas as pd
pd.set_option('display.max_colwidth', None)
import folium
import folium.plugins
import logging
import numpy as np

import shapely
logging.basicConfig(level=logging.INFO)

## Select an area of interest

- Select a drawing tool (polygon, circle or rectangle)
- Draw the AOI on the map
- Click "Export" on the right to save the GeoJSON file

In [ ]:
map_select = folium.Map()
folium.plugins.Draw(
    export=True,
    draw_options={
        "polyline": False,
        "circlemarker": False,
        "marker": False,
    },
).add_to(map_select)
map_select

## Search

In [3]:
# load a geometry
file_aoi = "/data/aoi_Maroc.geojson"
# file_aoi = "/data/aoi_bretagne.geojson"
gj_aoi = read_geojson(file_aoi)
aoi = geojson_to_wkt(gj_aoi["features"][0])
shp = gpd.read_file(file_aoi).geometry[0]

# open credentials
with open("/data/creds.json") as f:
    creds = json.load(f)

# search for products intersecting geometry
api = SentinelAPI(creds["username"], creds["password"])
products = api.query(
    aoi,
    platformname="Sentinel-2",
    date=("20230901", "20230920"),
    producttype="S2MSI1C",
    #  orbitdirection='ASCENDING',
    limit=50,
)

# make a geodataframe for visualization
gdf = api.to_geodataframe(products).reset_index(names='id')
dates = np.unique(gdf.beginposition.astype(str)).tolist()
print(f"Unique dates:\n{"\n".join(dates)}")

INFO:sentinelsat.SentinelAPI:Found 25 products


Unique dates:
2023-09-01 11:21:21.024
2023-09-03 11:06:29.024
2023-09-06 11:21:19.024
2023-09-08 11:06:21.024
2023-09-11 11:21:21.024
2023-09-13 11:06:39.024
2023-09-16 11:21:19.024
2023-09-18 11:07:21.024


In [4]:
visualize_S2_products(gdf, dates[7], gj_aoi)

## Download

In [ ]:
# fill indices of products to download
indices_to_download = [1, 5]
ids = gdf['id'][indices_to_download]
# dl = api.download_all_quicklooks(ids, directory_path='/data/S1/')
dl = api.download_all(ids, directory_path='/data/S1/')

In [19]:
from localtileserver import TileClient, get_folium_tile_layer
sel = gdf[gdf.beginposition == dates[1]]
dl = api.download_all_quicklooks(sel.id, directory_path='/data/tmp/')
# client = TileClient(file_in)
# t = get_folium_tile_layer(client, vmin=0.0, vmax=1.0)

# m = folium.Map(location=client.center(), zoom_start=client.default_zoom)
# t.add_to(m)

INFO:sentinelsat.SentinelAPI:Will download 4 quicklooks
INFO:sentinelsat.SentinelAPI:Downloading quicklook S2B_MSIL1C_20230903T110629_N0509_R137_T29RNQ_20230903T131433 to /data/tmp/S2B_MSIL1C_20230903T110629_N0509_R137_T29RNQ_20230903T131433.jpeg
INFO:sentinelsat.SentinelAPI:Downloading quicklook S2B_MSIL1C_20230903T110629_N0509_R137_T29RMP_20230903T131433 to /data/tmp/S2B_MSIL1C_20230903T110629_N0509_R137_T29RMP_20230903T131433.jpeg
INFO:sentinelsat.SentinelAPI:Downloading quicklook S2B_MSIL1C_20230903T110629_N0509_R137_T29RNP_20230903T131433 to /data/tmp/S2B_MSIL1C_20230903T110629_N0509_R137_T29RNP_20230903T131433.jpeg
INFO:sentinelsat.SentinelAPI:Downloading quicklook S2B_MSIL1C_20230903T110629_N0509_R137_T29RMQ_20230903T131433 to /data/tmp/S2B_MSIL1C_20230903T110629_N0509_R137_T29RMQ_20230903T131433.jpeg


In [1]:
from shapely.wkt import loads
import rasterio
from rasterio.merge import merge
from rasterio.transform import from_gcps, GroundControlPoint

# Create an empty list to store the opened images
opened_images = []

for _, item in dl[0].items():
    footprint = loads(item['footprint'])
    image_path = item['path']
    # with rasterio.open(image_path) as src:
    #     src.transform = rasterio.transform.from_origin(
    #         src.bounds.left, src.bounds.top, pixel_size, pixel_size
    #     )
    #     opened_images.append(src)

# Mosaic the opened images
# mosaic, out_transform = merge(opened_images)

# with rasterio.open('mosaic_geocoded_quicklooks.tif', 'w', driver='GTiff', width=mosaic.shape[2], height=mosaic.shape[1], count=mosaic.shape[0], dtype=mosaic.dtype, crs=src.crs, transform=out_transform) as dst:
#     dst.write(mosaic)

NameError: name 'dl' is not defined

In [35]:
item

{'id': '63e18dad-42eb-479d-969d-72e5e4dfa18b',
 'title': 'S2B_MSIL1C_20230903T110629_N0509_R137_T29RMQ_20230903T131433',
 'size': 521589195,
 'md5': 'e97f7fde29c621734b90c2af46b0f156',
 'date': datetime.datetime(2023, 9, 3, 11, 6, 29, 24000),
 'footprint': 'POLYGON((-9.681671 30.641939649077752,-9.64679 30.78065337291136,-9.608154 30.92830776313329,-9.570099 31.076171114267257,-9.531555 31.22391813119656,-9.4938965 31.37202843705144,-9.455444 31.519942774426347,-9.425629 31.633535170249402,-8.897064 31.635505628734723,-8.898132 30.644778742018026,-9.681671 30.641939649077752))',
 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('63e18dad-42eb-479d-969d-72e5e4dfa18b')/$value",
 'Online': False,
 'Creation Date': datetime.datetime(2023, 9, 3, 16, 48, 4, 384000),
 'Ingestion Date': datetime.datetime(2023, 9, 3, 16, 47, 39, 657000),
 'manifest_name': 'manifest.safe',
 'product_root_dir': 'S2B_MSIL1C_20230903T110629_N0509_R137_T29RMQ_20230903T131433.SAFE',
 'quicklook_url': "ht